In [13]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

import TropiGAT_functions 
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/ensemble_2809"

> Make model : 

In [5]:
dico_models = TropiGAT_functions.make_ensemble_TropiGAT(path_ensemble)

> Ferriol

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)

In [ ]:
dpo_embeddings

> Bea

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings.set_index([0], inplace = True)
bea_embeddings


> Towndsend

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings.set_index([0], inplace = True)

towndsend_embeddings

In [ ]:
towndsend_embeddings.info()

>Others old

In [4]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_phages.esm2.embedding.csv", sep = "," , header = None)
others_embeddings = others_embeddings.drop([1281] , axis = 1)
others_embeddings.set_index([0], inplace = True)

namesother_df = pd.read_csv(f"{path_project}/Others/index_others.tsv", sep = "\t" , names = ["index_phage", "index_prot","prot_name"])
new_index = []
for _,index in enumerate(others_embeddings.index) : 
    i_phage = int(index.split("__")[0])
    i_prot = int(index.split("__")[1])
    prot = namesother_df[(namesother_df["index_phage"] == i_phage) & (namesother_df["index_prot"] == i_prot)]["prot_name"].values[0]
    new_index.append(prot)

others_embeddings.index = new_index
others_embeddings.index.name = 0

> Others 

In [7]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_all.esm2.embedding.csv", sep = "," , header = None)
others_embeddings.set_index([0], inplace = True)

***
> run the predictions 

In [ ]:
# Run the predictions ferriol :
ferriol_predictions = {}
for dpo in dpo_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([dpo_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    ferriol_predictions[dpo] = pred

In [ ]:
ferriol_predictions

In [ ]:
# format the results : 
ferriol_pred_formated = TropiGAT_functions.format_predictions(ferriol_predictions , sep = "__")
TropiGAT_functions.clean_print(ferriol_pred_formated)

***

In [ ]:
# Run the predictions Bea :
bea_predictions = {}
for dpo in bea_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([bea_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    bea_predictions[dpo] = pred

In [ ]:
# format the results : 
bea_pred_formated = TropiGAT_functions.format_predictions(bea_predictions , sep = "_")
TropiGAT_functions.clean_print(bea_pred_formated)

In [ ]:
towndsend_embeddings.info()

In [ ]:
# Run the predictions Towndsend :
towndsend_predictions = {}
for dpo in towndsend_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([towndsend_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    towndsend_predictions[dpo] = pred

In [ ]:
towndsend_predictions

In [ ]:
# format the results : 
towndsend_pred_formated = TropiGAT_functions.format_predictions(towndsend_predictions , sep = "_")
TropiGAT_functions.clean_print(towndsend_pred_formated)

***

In [9]:
others_embeddings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56 entries, NC_025418.1_prot_YP_009098385.1_34 to ON146449.1_prot_UPW35138.1_1
Columns: 1280 entries, 1 to 1280
dtypes: float64(1280)
memory usage: 562.5+ KB


In [10]:
others_embeddings.index

Index(['NC_025418.1_prot_YP_009098385.1_34',
       'NC_029099.1_prot_YP_009226011.1_50', 'MT966873.1_prot_QOV05502.1_43',
       'NC_013649.2_prot_YP_003347651.1_57', 'AB716666.1_prot_BAP15736.1_24',
       'MK903728.1_prot_QDF14645.1_43', 'NC_031246.1_prot_YP_009302756.1_52',
       'MZ826764.1_prot_UCR74083.1_31', 'MW655991.1_prot_QUU29414.1_2',
       'AB897757.1_prot_BAQ02839.1_59', 'NC_031246.1_prot_YP_009302745.1_41',
       'AB897757.1_prot_BAQ02841.1_61', 'MZ826764.1_prot_UCR74085.1_33',
       'NC_025418.1_prot_YP_009098375.1_24', 'AB897757.1_prot_BAQ02844.1_64',
       'MK903728.1_prot_QDF14639.1_37', 'AB897757.1_prot_BAQ02843.1_63',
       'AB897757.1_prot_BAQ02838.1_58', 'MT542697.1_prot_QKY78353.1_44',
       'MZ826764.1_prot_UCR74084.1_32', 'ON146449.1_prot_UPW35150.1_13',
       'MK903728.1_prot_QDF14644.1_42', 'AB716666.1_prot_BAP15746.1_34',
       'MT542697.1_prot_QKY78347.1_38', 'MZ826764.1_prot_UCR74082.1_30',
       'MT966873.1_prot_QOV05496.1_37', 'MN781108.1_pro

In [14]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in enumerate(others_embeddings.index) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] :
        try : 
            graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
            pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
            other_predictions[dpo] = pred
        except Exception as e :
            print(e, dpo)

 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1
 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1


In [15]:
# format the results : 
others_pred_formated = TropiGAT_functions.format_predictions(other_predictions , sep = "_prot_")
TropiGAT_functions.clean_print(other_predictions)

{'AB716666.1_prot_BAP15736.1_24': {'KL110': 0.9716,
                                   'KL123': 0.9985,
                                   'KL125': 0.8789,
                                   'KL127': 0.5852,
                                   'KL14': 0.7169,
                                   'KL145': 0.8723,
                                   'KL18': 0.578,
                                   'KL28': 0.554,
                                   'KL3': 0.9214,
                                   'KL43': 0.5237,
                                   'KL57': 0.7509,
                                   'KL7': 0.9922,
                                   'KL70': 0.8128,
                                   'KL74': 0.9655,
                                   'KL8': 0.8561,
                                   'KL9': 0.9793},
 'AB716666.1_prot_BAP15746.1_34': {'KL111': 0.5726,
                                   'KL128': 0.9025,
                                   'KL149': 0.698,
                             

In [16]:
predictions = [other_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.Others.all.results.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

***
# Write the results : 

In [10]:
predictions = [ferriol_predictions , bea_predictions , towndsend_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.results.tsv", "w") as outfile:
    for prediction in predictions:
        for prot in prediction:
            if prediction[prot] == "No hits" or len(prediction[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

NameError: name 'ferriol_predictions' is not defined

> Parse TropiGAT and Seqbased results :

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean"

tropigat_results = pd.read_csv(f"{path_project}/TropiGAT.results.tsv", sep = "\t", names = ["protein", "predictions_tropigat"])
seqbased_results = pd.read_csv(f"{path_project}/Seqbased_model.results.tsv", sep = "\t", names = ["protein", "predictions_seqbased"])


In [ ]:
tropigat_results["protein_id"] = tropigat_results["protein"].apply(lambda x : x.split("_Dpo")[0])
seqbased_results["protein_id"] = seqbased_results["protein"].apply(lambda x : x.split("_A_")[0])

merged_df = tropigat_results.merge(seqbased_results, on='protein_id', how='inner')
merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])

merged_df_sorted = merged_df.sort_values(by='phage', ascending=True)
merged_df_sorted

In [ ]:
final_df = merged_df_sorted[["phage","protein_id", "predictions_seqbased", "predictions_tropigat"]]

final_df

In [ ]:
final_df.to_csv(f"{path_project}/PPT_results.tsv", sep = "\t", header = True, index = False)